In [20]:
import pandas as pd
import numpy as np
import random

In [16]:
%run Programming_project_cleaning_and_formatting.ipynb
%run getValue_functions_and_Initial_solution.ipynb

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Student ID        244 non-null    object
 1   Credits Required  244 non-null    int64 
 2   0                 244 non-null    object
 3   1                 244 non-null    object
 4   2                 244 non-null    object
 5   3                 244 non-null    object
 6   4                 244 non-null    object
 7   5                 244 non-null    object
 8   6                 244 non-null    object
 9   7                 244 non-null    object
dtypes: int64(1), object(9)
memory usage: 21.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course ID     46 non-null     object
 1   Min Students  46 non-null     int64 
 2   Max Students  46 

In [39]:
to_be_distributed = getAllStudents() #list of students that are to_be_distributed
randomized = random.sample(to_be_distributed, len(to_be_distributed))  #randomizing the list